<a href="https://colab.research.google.com/github/John-D-Boom/CompVision/blob/main/Testing_Loading_New_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from skimage import io, transform
from sklearn import metrics
import scipy
# import timm
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# from torchmetrics.classification import BinaryAUROC

PyTorch Version:  1.13.0+cu116
Torchvision Version:  0.14.0+cu116


In [2]:
!wget https://people.eecs.berkeley.edu/~hendrycks/imagenet-o.tar

--2023-01-10 01:39:52--  https://people.eecs.berkeley.edu/~hendrycks/imagenet-o.tar
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177617920 (169M) [application/x-tar]
Saving to: ‘imagenet-o.tar’

imagenet-o.tar      100%[===================>] 169.39M  35.4MB/s    in 5.3s    

2023-01-10 01:39:58 (31.9 MB/s) - ‘imagenet-o.tar’ saved [177617920/177617920]



In [3]:
!tar -xvf imagenet-o.tar -C imagenet-o

tar: imagenet-o: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [4]:
!pwd

/content


In [5]:
!ls

drive  imagenet-o.tar  sample_data


In [6]:
!tar xvf imagenet-o.tar 

imagenet-o/n03717622/stained-glass_window_0.99911314.JPEG
imagenet-o/n04141975/handcart_pushcart_cart_go-cart_0.95652604.JPEG
imagenet-o/n02948072/juice_0.9823741.JPEG
imagenet-o/n04371430/nude_nude_painting_0.99991727.JPEG
imagenet-o/n04023962/kitten_kitty_0.99977654.JPEG
imagenet-o/n01819313/redbud_Cercis_canadensis_0.9998136.JPEG
imagenet-o/n03482405/coffin_casket_0.9960121.JPEG
imagenet-o/n04418357/star_0.994911.JPEG
imagenet-o/n03724870/Homo_erectus_0.99577624.JPEG
imagenet-o/n03126707/
imagenet-o/n03297495/miller_milling_machine_0.9981591.JPEG
imagenet-o/n04371430/adonis_0.99964094.JPEG
imagenet-o/n03494278/
imagenet-o/n04542943/
imagenet-o/n02786058/
imagenet-o/n04423845/microfossil_0.98786455.JPEG
imagenet-o/n03530642/
imagenet-o/n07768694/jujube_Chinese_date_Chinese_jujube_0.9996817.JPEG
imagenet-o/n02783161/microscope_0.99072343.JPEG
imagenet-o/n04507155/paper_0.99870634.JPEG
imagenet-o/n03788365/person_individual_someone_somebody_mortal_soul_0.99961275.JPEG
imagenet-o/n03991

In [7]:
names_dict = {}
names_dict_paths = {}
all_paths = []
for folder in os.listdir('/content/imagenet-o'):
    if os.path.isfile(os.path.join('/content/imagenet-o', folder)):
        continue
    
    for file in os.listdir(os.path.join('/content/imagenet-o', folder)):
        print(file)
        all_paths.append(os.path.join(folder, file))
        broken_up = file.split(sep = "_")[:-1]
        class_name = '_'.join(broken_up)
        if class_name == '':
            print(file, broken_up)
        if class_name in names_dict.keys():
            names_dict[class_name] += 1
            names_dict_paths[class_name] += [os.path.join(folder, file)]
        else:
            names_dict[class_name] = 1
            names_dict_paths[class_name] = [os.path.join(folder, file)]

scatter_rug_throw_rug_0.99989474.JPEG
fabric_cloth_material_textile_0.999966.JPEG
necklet_0.99240243.JPEG
batiste_0.99971575.JPEG
shirttail_0.9939031.JPEG
argyle_argyll_0.99998033.JPEG
shirttail_0.99995387.JPEG
webbing_0.9997521.JPEG
tabi_tabis_0.9995814.JPEG
schoolbag_0.99998796.JPEG
camouflage_0.9996209.JPEG
olive_drab_0.9977106.JPEG
camouflage_0.99917763.JPEG
seabag_0.9999856.JPEG
canteen_0.9961816.JPEG
polo_shirt_sport_shirt_0.9846922.JPEG
brush_0.9975001.JPEG
temporary_hookup_patch_0.9974649.JPEG
coil_spring_volute_spring_0.99918336.JPEG
dendrite_0.93461937.JPEG
blastocyst_blastodermic_vessicle_0.98037934.JPEG
coil_spring_volute_spring_0.98507106.JPEG
chain_fern_0.9797577.JPEG
hedge_trimmer_0.9858503.JPEG
walking_fern_walking_leaf_Asplenium_rhizophyllum_Camptosorus_rhizophyllus_0.99983823.JPEG
cork_bottle_cork_0.98891664.JPEG
surge_suppressor_surge_protector_spike_suppressor_spike_arrester_lightning_arrester_0.99489105.JPEG
attenuator_0.98216486.JPEG
welder's_mask_0.99974424.JPEG


In [8]:
unseen_sim_train_arr = []
total_multiple = 0
total_combos = 0



for key in names_dict.keys():
    
    num_examples = names_dict[key]
    # print(num_examples)
    paths = names_dict_paths[key]
    if num_examples > 1:
        # print()
        total_multiple += 1
        total_combos += np.math.factorial(num_examples)/(2*np.math.factorial(num_examples-2))
        print(key, np.math.factorial(num_examples)/(2*np.math.factorial(num_examples-2)))
        for i in range(num_examples):
            for x in range(i+1, num_examples):
                path1 = paths[i]
                path2 = paths[x]
                label1 = key
                label2 = key
                unseen_sim_train_arr.append([path1, label1, path2, label2])
print(total_multiple)
print(total_combos)    
print(names_dict[""])

shirttail 1.0
camouflage 1.0
seabag 1.0
canteen 1.0
brush 1.0
temporary_hookup_patch 3.0
coil_spring_volute_spring 3.0
dendrite 1.0
blastocyst_blastodermic_vessicle 1.0
surge_suppressor_surge_protector_spike_suppressor_spike_arrester_lightning_arrester 1.0
attenuator 1.0
ethernet 1.0
flashlight_torch 3.0
potty_seat_potty_chair 1.0
flashlight_battery 1.0
cartridge_fuse 1.0
vibrator 1.0
jack 1.0
scouring_pad 1.0
wing_nut_wing-nut_wing_screw_butterfly_nut_thumbnut 3.0
brass 6.0
earplug 10.0
air_hammer_jackhammer_pneumatic_hammer 1.0
Cassegrainian_telescope_Gregorian_telescope 1.0
centrifugal_pump 1.0
motor 1.0
sprocket_sprocket_wheel 1.0
sugar_candy 6.0
ellipse_oval 1.0
bit 1.0
cookie_cutter 6.0
Cheops_Khufu 1.0
server_host 1.0
incubator_brooder 1.0
bow_bowknot 1.0
banner_streamer 1.0
star 10.0
tanager 1.0
candy_confect 3.0
electrolytic_electrolytic_capacitor_electrolytic_condenser 1.0
bullion 3.0
chalk 6.0
heart 3.0
cube_regular_hexahedron 3.0
window 1.0
wheel 1.0
batik 1.0
pinball_machi

KeyError: ignored

In [ ]:
print(len(unseen_sim_train_arr))

Ok now let's add 504 images from different classes



1.   For each unique class
2.   Grab a random other class
3.   Check that it's not the same. If not, make that a pairing.
4.   Repeat step 3 two more times so that there are 3 different samples for each.



In [ ]:
len(all_paths)
unseen_dif_train_arr = []
np.random.seed(0)
for num, key in enumerate(names_dict.keys()):
    print(key)
    path1 = names_dict_paths[key][0] #just considers the first one bc why not
    print(path1)
    label1 = key

    
    
    attempt = 1
    dif_labels_indices = []
    print("dif labels:", dif_labels_indices)
    for i in range(3):
        path2 = ""
        label2 = ""
        checked = False
        while not checked:
            
            attempt += 1
            random_idx = np.random.randint(0,1617)
            
            path2 = all_paths[random_idx]
            

            broken_up = path2.split(sep = "_")[:-1]
            broken_up = path2.split(sep = "/")[1]
            label2 = ''.join(broken_up)
            print(key, attempt, label2)
            if label2 != key and (random_idx not in dif_labels_indices):
                checked = True
                dif_labels_indices.append(random_idx)
                
        
        unseen_dif_train_arr.append([path1, label1, path2, label2])





In [ ]:
print(len(names_dict.keys()))

In [ ]:
imagenet_0_different_df = pd.DataFrame(unseen_dif_train_arr, columns = ["image1", "label1", "image2", "label2"])

In [ ]:
save_dif_path = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/test_imagenetO_different.csv'
imagenet_0_different_df.to_csv(save_dif_path, header = False, index = False)